In [ ]:
import os
import pandas as pd

# Path to the folder containing the CSV files
folder_path = 'My Files'

# File to keep track of processed files
processed_files_log = 'processed_files.txt'

# Initialize an empty list to hold the dataframes
df_list = []

# Check if the log file exists, if not, create an empty one
if not os.path.exists(processed_files_log):
    with open(processed_files_log, 'w') as log_file:
        log_file.write('')  # Create an empty file

# Read the log file to get the list of processed files
with open(processed_files_log, 'r') as log_file:
    processed_files = log_file.read().splitlines()

# Iterate over all files in the folder
for filename in os.listdir(folder_path):
    # Check if the file is a CSV file and has not been processed before
    if filename not in processed_files:
        # Create the full file path
        file_path = os.path.join(folder_path, filename)
        try:
            # Read the CSV file into a dataframe
            df = pd.read_csv(file_path, low_memory=False, encoding='latin1', on_bad_lines='skip')
            # Check if the dataframe is not empty
            if not df.empty:
                # Append the dataframe to the list
                df_list.append(df)
                # Add the filename to the processed files list
                processed_files.append(filename)
        except pd.errors.ParserError as e:
            print(f"Error reading {file_path}: {e}")

# Check if the list of dataframes is not empty
if df_list:
    # Concatenate all dataframes in the list into a single dataframe
    new_data = pd.concat(df_list, ignore_index=True)

    # Load the existing combined DataFrame if it exists
    if 'combined_df' in locals():  # Check if combined_df exists from previous runs
        combined_df = pd.concat([combined_df, new_data], ignore_index=True)
    else:
        combined_df = new_data

    # Display the first few rows of the combined dataframe
    print(combined_df.head())

    # Update the log file with the newly processed files
    with open(processed_files_log, 'w') as log_file:
        log_file.write('\n'.join(processed_files))
else:
    print("No new CSV files found.")


In [ ]:
# Create a copy of 'combined_df' to work with
df = combined_df.copy()

# Remove duplicate rows based on the 'Transaction ID' column
df = df.drop_duplicates(subset='Transaction ID')

# Convert the 'Date' column to datetime format, specifying the format of the date string
# Handle parsing errors by converting invalid parsing to NaT (Not a Time)
df['Date'] = pd.to_datetime(df['Date'], format='%d.%m.%y %H.%M.%S', errors='coerce')

# Extract the month name from the 'Date' column and create a new column 'Month Name'
df['Month Name'] = df['Date'].dt.month_name()


In [ ]:
import numpy as np

# Create a new column 'BIN' by extracting the first 6 characters from the 'PAN ' column
df['BIN'] = df['PAN '].str[:6]

# Create a new column 'Card Type' based on the 'BIN' values
# Use np.where to classify 'BIN' into 'Debit', 'Prepaid', or 'Credit'
df['Card Type'] = np.where(
    df['BIN'].isin(['000111', '000222', '000333', '000444']), 'Debit', 
    np.where(
        df['BIN'].isin(['111000', '111222', '111333']), 'Prepaid', 
        'Credit'
    )
)


In [ ]:
# Group data by 'Month Name' and 'Response code', then count the occurrences
grouped = grouped.groupby(['Month Name', 'Response code']).size().reset_index(name='Count')

# Pivot the table to get 'Month Name' as index, 'Response code' as columns, and 'Count' as values
# Fill missing values with 0 and transpose the table
pivot = grouped.pivot_table(index='Month Name', columns='Response code', values='Count', fill_value=0).T

# Calculate the total count for each response code across all months
total_counts = pivot.sum(axis=0)

# Calculate the percentage for each response code within each month
pivot_percentage = (pivot.divide(total_counts, axis=1) * 100)

# Create a copy of pivot_percentage to sort by each month
sorted_combined = pivot_percentage.copy()

# Iterate over each month column to sort the data by response code percentages in descending order
for month in sorted_combined.columns:
    sorted_combined = sorted_combined.sort_values(by=(month), ascending=False)

# Display the sorted data
sorted_combined


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Define custom colors for specific response codes
custom_colors = {
    '00 - approved': '#008000',
    '51-Insufficient funds': '#FF0000',# Example color for "51-Insufficient funds"
    # Add more response codes and colors if needed
}

# Remove duplicates and filter for September
grouped = df.drop_duplicates(subset='Transaction ID')
grouped = grouped[grouped['Month Name'] == 'September']

# Group by 'Card Type' and 'Response code'
grouped = grouped.groupby(['Card Type', 'Response code']).size().reset_index(name='Count')

# Calculate total counts per card type
total_counts = grouped.groupby('Card Type')['Count'].transform('sum')

# Calculate percentages of each response code per card type
grouped['Percentage'] = (grouped['Count'] / total_counts) * 100

# Filter out '00 - approved' and find the top 5 response codes per card type
# top_5 = grouped[grouped['Response code'] != '00 - approved']
top_5 = grouped.groupby('Card Type').apply(lambda x: x.nlargest(5, 'Percentage')).reset_index(drop=True)

# Loop through each Card Type and plot the top 5 response codes for that card type
for card_type in top_5['Card Type'].unique():
    # Filter the data for the current card type
    card_type_data = top_5[top_5['Card Type'] == card_type]

     # Create a list of colors based on the response code
    bar_colors = [custom_colors.get(rc, '#808080') for rc in card_type_data['Response code']]  # Default to gray if not specified
        
    # Create the plot
    plt.figure(figsize=(10, 7))
    ax = sns.barplot(x='Response code', y='Percentage', hue='Response code', data=card_type_data, palette=bar_colors, legend=False)
    
    # Rotate the x-axis labels to avoid clustering
    plt.xticks(rotation=45, ha='right')  # Rotate labels and align to the right

    # Extend y-axis limits to provide space for labels
    #plt.ylim(0, top_5['Percentage'].max() * 1.1)  # Add 20% space above the highest bar

    # Add percentage labels on top of the bars
    for p in ax.patches:
        height = p.get_height()
        ax.annotate(f'{height:.2f}%', 
                    (p.get_x() + p.get_width() / 2., height), 
                    ha='center', va='bottom', 
                    xytext=(0, 3), textcoords='offset points', fontsize=10, color='black')
    
    # Add title and labels
    plt.title(f'Top 5 Response Codes for {card_type}')
    plt.xlabel('Response Code')
    plt.ylabel('Percentage (%)')
    
    # Show the plot
    plt.tight_layout()
    plt.show()
